In [ ]:
import os
import json
from sklearn import preprocessing
from sklearn.naive_bayes import GaussianNB
from sklearn import preprocessing
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB

def fetchTrainingData():
    trainingDataPath = os.getcwd() + "/training.json"
    with open(trainingDataPath) as trainingFile:
        lines = trainingFile.readlines()
    numberOfLines = int(lines[0])
    topic = []
    questionAndExcerpt = []
    excerpt = []
    for i in range(1, numberOfLines):
        dataLine = lines[i]
        topic.append(readJson(dataLine, "topic"))
        questionAndExcerpt.append(readJson(dataLine, "question") + readJson(dataLine, "excerpt"))
    return topic, questionAndExcerpt
        
def readJson(data, key):
    return json.loads(data)[key]

topic, questionAndExcerpt = fetchTrainingData()

labelEncoder = preprocessing.LabelEncoder()
encodedTopic = labelEncoder.fit_transform(topic)

countVectoriser = CountVectorizer(stop_words="english")
X_train_counts = countVectoriser.fit_transform(questionAndExcerpt)

tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

clf = MultinomialNB().fit(X_train_tfidf, encodedTopic)

def predict(data, model = clf):
    X_new_counts = countVectoriser.transform(data)
    X_new_tfidf = tfidf_transformer.transform(X_new_counts)      
    predicted = model.predict(X_new_tfidf)
   
    for category in labelEncoder.inverse_transform(predicted):
        print(category)

testDataCount = int(input())
testData = []
for i in range(0, testDataCount):
    dataLine = input()
    testData.append(readJson(dataLine, "question") + readJson(dataLine, "excerpt"))

predict(testData)